In [1]:
!git clone https://github.com/speckean/upar_dataset.git

fatal: destination path 'upar_dataset' already exists and is not an empty directory.


# Import Tools

In [2]:
import os
import pickle
import glob
import random

import numpy as np
import torch.utils.data as data
from PIL import Image

import torchvision.transforms as T
from torchvision import models

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [3]:
HOME = os.getcwd()
HOME

'C:\\Users\\USER\\Downloads\\LabML\\FashionDetection(Week3)\\UPAR(Soft-Biometric-Classification)'

# Dataset

## Get a data

In [4]:
!git clone https://github.com/speckean/upar_challenge.git
%cd upar_challenge
!pip install tqdm gdown requests easydict

C:\Users\USER\Downloads\LabML\FashionDetection(Week3)\UPAR(Soft-Biometric-Classification)\upar_challenge


fatal: destination path 'upar_challenge' already exists and is not an empty directory.


In [5]:
!python download_datasets.py
%cd {HOME}
!move upar_challenge/data .

Download Market 1501 dataset
Extract Market 1501 dataset
C:\Users\USER\Downloads\LabML\FashionDetection(Week3)\UPAR(Soft-Biometric-Classification)


Downloading...
From (original): https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM
From (redirected): https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM&confirm=t&uuid=1c7aa9b8-2cfc-41e1-ba45-db8dcd2e6ac3
To: C:\Users\USER\Downloads\LabML\FashionDetection(Week3)\UPAR(Soft-Biometric-Classification)\upar_challenge\data\market_1501.zip

  0%|          | 0.00/153M [00:00<?, ?B/s]
  0%|          | 524k/153M [00:00<00:46, 3.29MB/s]
  2%|1         | 2.62M/153M [00:00<00:12, 11.6MB/s]
  3%|3         | 4.72M/153M [00:00<00:19, 7.70MB/s]
  5%|5         | 7.86M/153M [00:00<00:11, 12.5MB/s]
  7%|7         | 11.0M/153M [00:00<00:08, 16.4MB/s]
  9%|9         | 14.2M/153M [00:00<00:06, 20.1MB/s]
 11%|#1        | 17.3M/153M [00:01<00:05, 23.0MB/s]
 14%|#3        | 21.0M/153M [00:01<00:05, 26.1MB/s]
 16%|#6        | 24.6M/153M [00:01<00:04, 28.0MB/s]
 19%|#8        | 28.3M/153M [00:01<00:04, 29.3MB/s]
 21%|##        | 31.5M/153M [00:01<00:04, 29.7MB/s]
 23%|##2       | 35.1M/153M [00:

In [6]:
current_dir_path = 'data/PA100k/release_data/release_data'
new_dir_path = 'data/PA100k/data'


if os.path.exists(current_dir_path):
    if not os.path.exists(new_dir_path):
        os.rename(current_dir_path, new_dir_path)
        print(f"Directory renamed from '{current_dir_path}' to '{new_dir_path}'")

    else:
        print(f"The target directory '{new_dir_path}' already exists.")
else:
    print(f"The directory '{current_dir_path}' does not exist.")

The directory 'data/PA100k/release_data/release_data' does not exist.


## Construct dataset/dataloader

In [7]:

class UPAR(data.Dataset):
  """
  Load UPAR dataset from pickle file

  split: whether to use train/val/trainval/test split
  partition: partition id 0-9
  root: path to datasets, original datasets must be in this directory
  data_path: path to UPAR pickle file
  transform: training data transforms
  target_transforms: evaluation data transforms
  """
  def __init__(self, split='train', partition=0, root=HOME, data_path='upar_dataset/UPAR/dataset_all.pkl', transform=None, target_transform=None):
    dataset_info = pickle.load(open(data_path, 'rb+'))
    self.dataset_info = dataset_info

    img_id = dataset_info.image_name
    attr_label = dataset_info.label

    assert split in dataset_info.partition.keys(), f'split {split} does not exist'

    self.dataset = 'UPAR'
    self.transform = transform  # data transforms during training
    self.target_transform = target_transform  # data transforms during testing

    self.root_path = root+"/data"  # path to datasets

    self.attr_id = dataset_info.attr_name  # attribute names
    self.attr_num = len(self.attr_id)  # number of attributes

		# load partition
    self.img_idx = dataset_info.partition[split]

    if isinstance(self.img_idx, list):
      self.img_idx = self.img_idx[partition]

    if isinstance(self.img_idx, list):
      self.img_idx = np.hstack(self.img_idx)

    self.img_idx = np.array([i for i in self.img_idx if not any(folder in img_id[i] for folder in ["RAP"])])


    self.img_num = self.img_idx.shape[0]
    self.img_id = [img_id[i] for i in self.img_idx]
    self.label = attr_label[self.img_idx]

    # set sub-dataset lengths to enable evaluation on sub-datasets
    self.sub_dataset_lengths = [len(d) for d in dataset_info.partition.test[partition]]


  def __getitem__(self, index):
      """
      get dataset item by index

      index: item index
      return: image data (img) with corresponding ground truth labels (gt_label), dataset id (did), and image path (imgname)
      """
      imgname, gt_label, imgidx = self.img_id[index], self.label[index], self.img_idx[index]
      did = self.dataset_info.dataset_ids[imgidx]
      imgpath = os.path.join(self.root_path, imgname)
      img = Image.open(imgpath)

      if self.transform is not None:
          img = self.transform(img)

      gt_label = gt_label.astype(np.float32)

      if self.target_transform is not None:
          gt_label = self.transform(gt_label)

      return img, gt_label, did, imgname


  def __len__(self):
    """
    get length of dataset
    """
    return len(self.img_id)


def get_transform(height, width):
    normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    train_transform = [
        T.Resize((height, width))
    ]

    train_transform += [
        T.Pad(10),
        T.RandomCrop((height, width)),
        T.RandomHorizontalFlip(),
    ]

    train_transform += [
        T.ToTensor(),
        normalize,
    ]
    train_transform = T.Compose(train_transform)

    valid_transform = T.Compose([
        T.Resize((height, width)),
        T.ToTensor(),
        normalize
    ])

    return train_transform, valid_transform

In [8]:
height = 224
width = 224
train_transform, valid_transform = get_transform(height, width)


train_dataset = UPAR(split='train', partition=0, transform=train_transform)
val_dataset = UPAR(split='val', partition=0, transform=valid_transform)
test_dataset = UPAR(split='test', partition=0, transform=valid_transform)


In [9]:
batch_size = 32 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model Baseline

In [10]:
model = models.mobilenet_v2(pretrained=True)

C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
num_classes = 40
model.classifier[1] = nn.Sequential(
    nn.Linear(model.last_channel, num_classes),
    nn.Sigmoid()
)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [12]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    for batch, (X, y, _, _) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch % 100 == 0:
            print(f"Batch {batch}, Loss: {loss.item()}")

    avg_loss = total_loss / len(dataloader)
    print(f"Average Training Loss: {avg_loss}")
    return avg_loss

def validate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X, y, _, _ in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Average Validation Loss: {avg_loss}")
    return avg_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 100

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    print("-" * 10)
    train_loss = train_one_epoch(model, train_loader, loss_fn, optimizer, device)
    val_loss = validate(model, val_loader, loss_fn, device)
    scheduler.step()


Epoch 1/100
----------
Batch 0, Loss: 25.841175079345703
Batch 100, Loss: 24.357088088989258
Batch 200, Loss: 21.19321060180664
Batch 300, Loss: 21.68756103515625
Batch 400, Loss: 20.53506851196289
Batch 500, Loss: 22.070159912109375
Batch 600, Loss: 22.718292236328125
Batch 700, Loss: 21.468585968017578
Batch 800, Loss: 22.594327926635742
Batch 900, Loss: 20.596675872802734
Batch 1000, Loss: 21.22633171081543
Batch 1100, Loss: 22.505311965942383
Batch 1200, Loss: 22.17971420288086
Batch 1300, Loss: 20.360164642333984
Batch 1400, Loss: 21.76951789855957
Batch 1500, Loss: 20.807865142822266
Batch 1600, Loss: 20.776824951171875
Batch 1700, Loss: 22.453277587890625
Batch 1800, Loss: 21.05763053894043
Batch 1900, Loss: 21.46938705444336
Batch 2000, Loss: 21.30282211303711
Batch 2100, Loss: 21.58887481689453
Batch 2200, Loss: 21.445358276367188
Batch 2300, Loss: 21.745054244995117
Batch 2400, Loss: 20.860084533691406
Batch 2500, Loss: 21.121145248413086
Batch 2600, Loss: 20.813039779663086


In [18]:
torch.save(model.state_dict(), "mobilenet_model.pth")

# Inference

In [14]:
def predict(model, dataloader, device, threshold=0.5):
    model.eval()
    predictions = []
    with torch.no_grad():
        for X, _, _, _ in dataloader:
            X = X.to(device)
            pred = model(X)
            pred = (pred > threshold).float()  # Convert probabilities to binary predictions
            predictions.append(pred.cpu().numpy())
    return np.vstack(predictions)


In [15]:
test_predictions = predict(model, test_loader, device)

In [17]:
test_predictions

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
# 'attr_name': ['Age-Young',
#   'Age-Adult',
#   'Age-Old',
#   'Gender-Female',
#   'Hair-Length-Short',
#   'Hair-Length-Long',
#   'Hair-Length-Bald',
#   'UpperBody-Length-Short',
#   'UpperBody-Color-Black',
#   'UpperBody-Color-Blue',
#   'UpperBody-Color-Brown',
#   'UpperBody-Color-Green',
#   'UpperBody-Color-Grey',
#   'UpperBody-Color-Orange',
#   'UpperBody-Color-Pink',
#   'UpperBody-Color-Purple',
#   'UpperBody-Color-Red',
#   'UpperBody-Color-White',
#   'UpperBody-Color-Yellow',
#   'UpperBody-Color-Other',
#   'LowerBody-Length-Short',
#   'LowerBody-Color-Black',
#   'LowerBody-Color-Blue',
#   'LowerBody-Color-Brown',
#   'LowerBody-Color-Green',
#   'LowerBody-Color-Grey',
#   'LowerBody-Color-Orange',
#   'LowerBody-Color-Pink',
#   'LowerBody-Color-Purple',
#   'LowerBody-Color-Red',
#   'LowerBody-Color-White',
#   'LowerBody-Color-Yellow',
#   'LowerBody-Color-Other',
#   'LowerBody-Type-Trousers&Shorts',
#   'LowerBody-Type-Skirt&Dress',
#   'Accessory-Backpack',
#   'Accessory-Bag',
#   'Accessory-Glasses-Normal',
#   'Accessory-Glasses-Sun',
#   'Accessory-Hat'],

#  ...],
#  'label': array([[0, 1, 0, ..., 1, 0, 0],
#         [0, 1, 0, ..., 0, 0, 0],
#         [0, 1, 0, ..., 1, 0, 0],
#         ...,
#         [0, 1, 0, ..., 0, 0, 0],
#         [0, 1, 0, ..., 0, 0, 0],
#         [0, 1, 0, ..., 0, 0, 0]]),